#### Normaliza o texto [transcrições]

In [ ]:
from Funcs.tratamentoDeTexto import normalize_text
from Funcs.getPaths import get_Paths
from Funcs.db import get_languange, get_data, update_data

from Funcs.db import new_collumn
new_collumn('videos_data.db','transcribes','transcription_normalizado','TEXT')

paths = get_Paths('Videos')
for path in paths:
    id = path.split("\\")[-1]
    normal = normalize_text(get_data('videos_data.db','transcribes','transcription',id)[0][1],get_languange('videos_data.db','links',id)[0])
    update_data('videos_data.db','transcribes','transcription_normalizado',id,normal)





#### Normaliza o texto [captions]

In [ ]:
from Funcs.tratamentoDeTexto import normalize_text
from Funcs.getPaths import get_Paths
from Funcs.db import get_data, update_data

from Funcs.db import new_collumn
new_collumn('videos_data.db','captions','captions_normalizado','TEXT')

paths = get_Paths('Videos')
for path in paths:
    id = path.split("\\")[-1]
    frames = get_data('videos_data.db','captions','caption',id)
    for frame in frames:
        #print('id: '+frame[0])
        #print('caption: '+frame[1])
        normal = normalize_text(frame[1],'en')
        update_data('videos_data.db','captions','captions_normalizado',frame[0],normal)
    






#### Vetorizar

In [30]:
# Adicionar todas as strings em uma única variável

from Funcs.db import get_data, push_data
from Funcs.getPaths import get_Paths
from sklearn.feature_extraction.text import TfidfVectorizer


#all_captionsNids = []
ids = []
textos = []
paths = get_Paths('Videos')
for path in paths:
    id = path.split("\\")[-1]   # Videos\\<id>
    
    frames_with_id = get_data('videos_data.db','captions','captions_normalizado',id)    # Retorna um conjunto de frames do video <id> o id do frame + a caption normalizada 
    aux_ids, aux_captions = zip(*frames_with_id)

    for id in aux_ids:
        ids.append(id)
    
    for text in aux_captions:
        textos.append(text)

vectorizer = TfidfVectorizer()
vetores = vectorizer.fit_transform(textos)

vectors_with_ids = {id_: vetores[idx] for idx, id_ in enumerate(ids)}

for frame in vectors_with_ids:
    Videoid = frame.split("_")[0]
    push_data('videos_data.db','textos_vetorizados','vetor',frame,str(vectors_with_ids[frame].toarray().tolist()),Videoid)

#### Vetorizar [transcrições]

In [7]:
# Adicionar todas as strings em uma única variável

from Funcs.db import get_data, push_data
from Funcs.getPaths import get_Paths
from sklearn.feature_extraction.text import TfidfVectorizer


#all_captionsNids = []
ids = []
textos = []
paths = get_Paths('Videos')
for path in paths:
    id = path.split("\\")[-1]   # Videos\\<id>
    
    trancript_with_id = get_data('videos_data.db','transcribes','transcription_normalizado',id)    # Retorna um conjunto de transcrições com id
    aux_ids, aux_transcript = zip(*trancript_with_id)
   
    ids.append(aux_ids[0])
    textos.append(aux_transcript[0])

vectorizer = TfidfVectorizer()
vetores = vectorizer.fit_transform(textos)

vectors_with_ids = {id_: vetores[idx] for idx, id_ in enumerate(ids)}

for frame in vectors_with_ids:
    push_data('videos_data.db','textos_vetorizados','vetor',frame,str(vectors_with_ids[frame].toarray().tolist()),frame)